<a href="https://colab.research.google.com/github/sai-ranganadh/Customer-Support-FAQ-system/blob/main/RAG_faqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
!pip install transformers torch faiss-cpu

In [20]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [2]:
with open('/content/drive/MyDrive/Ecommerce_FAQ_Chatbot_dataset.json') as faq_file:
  faq_data = json.load(faq_file)

In [3]:
questions = [faq_data['question'] for faq_data in faq_data['questions']]
answers = [faq_data['answer'] for faq_data in faq_data['questions']]

In [4]:
print(questions[:5])
print(answers[:5])

['How can I create an account?', 'What payment methods do you accept?', 'How can I track my order?', 'What is your return policy?', 'Can I cancel my order?']
["To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.", 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.', "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.", 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.', 'You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.']


In [5]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def generate_embeddings(texts):
  inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
    # Mean pooling to get sentence embeddings
  embeddings = outputs.last_hidden_state.mean(dim=1)
  return embeddings.numpy()

In [8]:
question_embeddings = generate_embeddings(questions)

In [9]:
import numpy as np
import faiss

embedding_dim = question_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(np.array(question_embeddings))

faq_answers = {i: answers[i] for i in range(len(answers))}
print(faq_answers)


{0: "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.", 1: 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.', 2: "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.", 3: 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.', 4: 'You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.', 5: 'Shipping times vary depending on the destination and the shipping method chosen. Standard shipping usually takes 3-5 business days, while express shipping ca

In [10]:
def get_answer(user_query):
  query_embedding = generate_embeddings([user_query])
  _, indices = index.search(query_embedding, k=1)
  answer_index = indices[0][0]
  return faq_answers[answer_index]

get_answer("What is the return policy?")

'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'

In [11]:
def get_answer(user_query):
  query_embedding = generate_embeddings([user_query])
  distances, indices = index.search(query_embedding, k=3)
  print(distances)
  print(indices)
  answers = [faq_answers[i] for i in indices[0]]
  return answers
  #return faq_answers[answer_index]

get_answer("What is the return policy?")

[[23.704485 29.966068 33.565384]]
[[ 3 45 63]]


['Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.',
 'Yes, you can return a product purchased during a sale or with a discount. The refund will be processed based on the amount paid after the discount.',
 'Clearance or final sale items are typically non-returnable and non-refundable. Please review the product description or contact our customer support team for more information.']

In [ ]:
import openai
import os
import time

openai.api_key = "your-openai-key" # Replace YOUR_API_KEY with your actual API key

def generate_response(user_query):
    retrieved_answer = get_answer(user_query)

    # Construct prompt
    prompt = f"""
    User asked: "{user_query}"
    Based on the FAQ, here is the related answer: "{retrieved_answer}"
    Provide a helpful and conversational response to the user.
    """

    # Use GPT model to generate the final response
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=150,
        temperature=0.7
    )
    time.sleep(5)

    return response['choices'][0]['text'].strip()

generate_response("What is the return policy?")